In [1]:
import pandas as pd
import string
import datetime
from datetime import datetime
from datetime import timedelta
import numpy as np
from tqdm import tqdm

from random import random
import os

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Here path should be the location of the git repo
path = "/home/skand/Workspace/MIT/analytics_edge/Analytics_Edge_Tennis_prediction"

In [3]:
os.chdir(path)
print(os.path.abspath(os.curdir))

/home/skand/Workspace/MIT/analytics_edge/Analytics_Edge_Tennis_prediction


# First create a dictionnary with all tournaments from 1991 to 2020

For each tournament, we have the following information: 
- Tourney year
- Tourney dates
- Number of players for each tournament
- A unique id to describe each tournament



In [4]:
#Data Importation
df_tournois=pd.read_csv('raw_data/tournaments_1877-2017_unindexed_csv.csv')
df_tournois=df_tournois[df_tournois['tourney_year']>=1991]
dict_tournoi=df_tournois[['tourney_year','tourney_dates','tourney_slug','tourney_name','tourney_conditions','tourney_location','tourney_singles_draw','tourney_surface']]
dict_tournoi['tourney_dates']=pd.to_datetime(dict_tournoi['tourney_dates'])

#We don't have data for years from 2018 to 2020, let's assume that tournament haven't change in these 3 years
## In 2016 and 2017, the number of tournaments did not change (conforts the assumption)
#We can then add lines corresponding to tournaments from 2018 to 2020
tournois_2017=dict_tournoi[dict_tournoi['tourney_year']==2017]
k=1
for year in range (2018,2021):
    df_year=tournois_2017
    # here modifying df_year also modifies tournois_2017
    df_year['tourney_year']=year
    df_year['tourney_dates']=pd.to_datetime(df_year['tourney_dates'])    
    df_year['tourney_dates']=df_year['tourney_dates']+timedelta(days=365)
    k+=1
    dict_tournoi=pd.concat([dict_tournoi,df_year])
dict_tournoi['tourney_dates']=dict_tournoi['tourney_dates'].apply(lambda x:x.strftime('%Y-%m-%d'))

#Create a function to avoid problems with location formatin over time
def change_loc(location):
    location=str(location).split(',')[0]
    return location

dict_tournoi['tourney_location']=dict_tournoi['tourney_location'].apply(lambda x:change_loc(x)) #To investigate: this changes (reduces) the number of unique tourney_location
dict_tournoi['tournoi_unique_id']=dict_tournoi['tourney_year'].astype(str)+'-'+dict_tournoi['tourney_location'].apply(lambda x:str(x).lower())+'-'+dict_tournoi['tourney_surface'].apply(lambda x:str(x).lower())
list_not_unique_2=dict_tournoi['tournoi_unique_id'].value_counts()[dict_tournoi['tournoi_unique_id'].value_counts()>1]
dict_tournoi['unique_slug']=dict_tournoi['tourney_year'].apply(lambda x:str(x))+'-'+dict_tournoi['tourney_slug']
dict_tournoi=dict_tournoi.drop('tourney_slug',axis=1).reset_index(drop=True)


#Here some 'tournoi_unique_id' are not unique, mainly because of Queens and wimbledon
list_not_unique=dict_tournoi['tournoi_unique_id'].value_counts()[dict_tournoi['tournoi_unique_id'].value_counts()>1]
dict_tournoi.loc[dict_tournoi['tournoi_unique_id'].isin(list_not_unique.index),'tournoi_unique_id']=dict_tournoi.loc[dict_tournoi['tournoi_unique_id'].isin(list_not_unique.index)]['tournoi_unique_id']+'-'+dict_tournoi.loc[dict_tournoi['tournoi_unique_id'].isin(list_not_unique.index)]['tourney_singles_draw'].apply(lambda x:str(x))
list_not_unique_2=dict_tournoi['tournoi_unique_id'].value_counts()[dict_tournoi['tournoi_unique_id'].value_counts()>1]
dict_tournoi=dict_tournoi[~dict_tournoi['tournoi_unique_id'].isin(list_not_unique_2.index)] # dropping 4 unique IDs in early 90s 
dict_tournoi.head()

,tourney_year,tourney_dates,tourney_name,tourney_conditions,tourney_location,tourney_singles_draw,tourney_surface,tournoi_unique_id,unique_slug
0,1991,1990-12-31,Adelaide,Outdoor,Adelaide,32,Hard,1991-adelaide-hard,1991-adelaide
1,1991,1990-12-31,Wellington,Outdoor,Wellington,32,Hard,1991-wellington-hard,1991-wellington
2,1991,1991-01-07,Auckland,Outdoor,Auckland,32,Hard,1991-auckland-hard,1991-auckland
3,1991,1991-01-07,Sydney Outdoor,Outdoor,Sydney,32,Hard,1991-sydney-hard-32,1991-sydney
4,1991,1991-01-14,Australian Open,Outdoor,Melbourne,128,Hard,1991-melbourne-hard,1991-australian-open


# Games 1991-2017

In [5]:
match_91_16=pd.read_csv('raw_data/match_scores_1991-2016_unindexed_csv.csv')
match_17=pd.read_csv('raw_data/match_scores_2017_unindexed_csv.csv')
match_91_17=pd.concat([match_91_16,match_17])
match_91_17['games_played']=match_91_17['winner_games_won']+match_91_17['loser_games_won']
match_91_17=match_91_17[['tourney_year_id','tourney_slug','tourney_round_name','round_order','winner_name','loser_name','games_played']]


def create_year(year_id):
    year_id=year_id.split('-')
    return int(year_id[0])

match_91_17['tourney_year_id']=match_91_17['tourney_year_id'].apply(lambda x:create_year(x))
match_91_17['unique_slug']=match_91_17['tourney_year_id'].apply(lambda x:str(x))+'-'+match_91_17['tourney_slug']

match_91_17=pd.merge(match_91_17,dict_tournoi,on='unique_slug')
match_91_17['tourney_dates']=pd.to_datetime(match_91_17['tourney_dates'])
match_91_17=match_91_17[['tourney_year','tourney_dates','round_order','winner_name','loser_name','games_played','tourney_conditions','tourney_surface','tourney_singles_draw','tournoi_unique_id']]
match_91_17=match_91_17.rename(columns={'tourney_year':'year'
                                        ,'tourney_dates':'date'
                                        ,'round_order':'round'
                                        ,'winner_name':'winner'
                                       , 'loser_name':'loser'
                                       ,'tourney_conditions':'court'
                                       , 'tourney_surface':'surface'
                                       })

#Change players name, to make same in the same format than other table
def change_player(player):
    player=player.split(' ')
    player=' '.join(player[1:])+'_'+player[0][0]+'.'
    return player

match_91_17['loser']=match_91_17['loser'].apply(lambda x:change_player(x).lower())
match_91_17['winner']=match_91_17['winner'].apply(lambda x:change_player(x).lower())

#Let's remove games whose date is juste before the beginning of the year
match_91_17=match_91_17[match_91_17['year']==match_91_17['date'].dt.year]

match_91_17['week_title']=match_91_17['tournoi_unique_id'].map(dict_tournoi[['tourney_dates','tournoi_unique_id']].set_index('tournoi_unique_id')['tourney_dates'])

#Creation of unique_ids pour players
match_91_17['id_winner']=(match_91_17['winner']+match_91_17['week_title']).apply(lambda x:x.replace(' ',''))
match_91_17['id_loser']=(match_91_17['loser']+match_91_17['week_title']).apply(lambda x:x.replace(' ',''))

match_91_17=match_91_17.drop('date',axis=1)

match_91_17

,year,round,winner,loser,games_played,court,surface,tourney_singles_draw,tournoi_unique_id,week_title,id_winner,id_loser
62,1991,1,novacek_k.,fleurian_j.,26,Outdoor,Hard,32,1991-auckland-hard,1991-01-07,novacek_k.1991-01-07,fleurian_j.1991-01-07
63,1991,2,novacek_k.,vajda_m.,26,Outdoor,Hard,32,1991-auckland-hard,1991-01-07,novacek_k.1991-01-07,vajda_m.1991-01-07
64,1991,2,fleurian_j.,mattar_l.,28,Outdoor,Hard,32,1991-auckland-hard,1991-01-07,fleurian_j.1991-01-07,mattar_l.1991-01-07
65,1991,3,fleurian_j.,sanchez_e.,21,Outdoor,Hard,32,1991-auckland-hard,1991-01-07,fleurian_j.1991-01-07,sanchez_e.1991-01-07
66,1991,3,novacek_k.,jonsson_l.,0,Outdoor,Hard,32,1991-auckland-hard,1991-01-07,novacek_k.1991-01-07,jonsson_l.1991-01-07
...,...,...,...,...,...,...,...,...,...,...,...,...
102506,2017,3,thiem_d.,carreno busta_p.,28,Indoor,Hard,8,2017-london-hard,2017-11-12,thiem_d.2017-11-12,carrenobusta_p.2017-11-12
102507,2017,3,sock_j.,cilic_m.,33,Indoor,Hard,8,2017-london-hard,2017-11-12,sock_j.2017-11-12,cilic_m.2017-11-12
102508,2017,3,dimitrov_g.,carreno busta_p.,14,Indoor,Hard,8,2017-london-hard,2017-11-12,dimitrov_g.2017-11-12,carrenobusta_p.2017-11-12
102509,2017,3,dimitrov_g.,goffin_d.,14,Indoor,Hard,8,2017-london-hard,2017-11-12,dimitrov_g.2017-11-12,goffin_d.2017-11-12


Now we will add the rank of each player during the games

# Games 2018-2020

In [6]:
match_2018_2020=pd.DataFrame()
for year in range (2018,2021):
    match_2018_2020=pd.concat([match_2018_2020,pd.read_excel('raw_data/'+str(year)+'.xlsx')],sort=False)
match_2018_2020=pd.concat([match_2018_2020[match_2018_2020.columns[0:27]],match_2018_2020[match_2018_2020.columns[-2:]]],axis=1)
match_2018_2020['games_played']=match_2018_2020[['W1','W2','W3','W4','W5','L1','L2','L3','L4','L5']].fillna(0).sum(axis=1)
match_2018_2020=match_2018_2020[['Location','Tournament','Date','Round','Court','Surface','Winner','Loser','games_played']]

match_2018_2020['Date']=pd.to_datetime(match_2018_2020['Date'])
match_2018_2020['year']=pd.to_datetime(match_2018_2020['Date']).dt.year

#We create a unique key for each tournament, in order to merge it with tournaments dictionnary
match_2018_2020['tournoi_unique_id']=match_2018_2020['year'].apply(lambda x:str(x))+'-'+match_2018_2020['Location'].apply(lambda x:x.lower())+'-'+match_2018_2020['Surface'].apply(lambda x:x.lower())

# We need to change the format of round for this matchs: 1:final, 2:1/2 final, etc. 
#But there are not the same amount of round in each tournament, this is why it is required to do a special change for each tournament
def change_round(Round):
    if Round=='1st Round':
        return 1
    if Round=='2nd Round':
        return 2
    if Round=='3rd Round':
        return 3
    if Round=='4th Round':
        return 4
    if Round=='Quarterfinals':
        return 5
    if Round=='Semifinals':
        return 6
    if Round=='The Final':
        return 7

match_2018_2020['Round']=match_2018_2020['Round'].apply(lambda x:change_round(x))

for tournoi in match_2018_2020['tournoi_unique_id'].value_counts().index:
    serie=match_2018_2020[match_2018_2020['tournoi_unique_id']==tournoi]['Round']
    sorting=serie.sort_values(ascending=False)
    mapping=sorting.value_counts().index.sort_values(ascending=False)
    nb_round=len(mapping)
    serie_2=pd.Series([k for k in range(1,nb_round+1)],index=mapping)
    match_2018_2020.loc[match_2018_2020['tournoi_unique_id']==tournoi,'Round']=match_2018_2020[match_2018_2020['tournoi_unique_id']==tournoi]['Round'].replace(serie_2) 

match_2018_2020=pd.merge(match_2018_2020,dict_tournoi,on='tournoi_unique_id')
match_2018_2020=match_2018_2020[['Date','year','Round','Court','Surface','Winner','Loser','tourney_singles_draw','tournoi_unique_id','games_played']]
match_2018_2020=match_2018_2020.rename(str.lower,axis='columns')
match_2018_2020['winner']=match_2018_2020['winner'].apply(lambda x:x.replace(' ','_').lower())
match_2018_2020['loser']=match_2018_2020['loser'].apply(lambda x:x.replace(' ','_').lower())

match_2018_2020['week_title']=match_2018_2020['tournoi_unique_id'].map(dict_tournoi[['tourney_dates','tournoi_unique_id']].set_index('tournoi_unique_id')['tourney_dates'])

match_2018_2020['id_winner']=match_2018_2020['winner']+match_2018_2020['week_title']
match_2018_2020['id_winner']=match_2018_2020['id_winner'].apply(lambda x:x.replace(' ',''))
match_2018_2020['id_loser']=(match_2018_2020['loser']+match_2018_2020['week_title']).apply(lambda x:x.replace(' ',''))

match_2018_2020=match_2018_2020.drop('date',axis=1)

match_2018_2020

,year,round,court,surface,winner,loser,tourney_singles_draw,tournoi_unique_id,games_played,week_title,id_winner,id_loser
0,2017,1.0,Outdoor,Hard,dolgopolov_o.,schwartzman_d.,28,2017-brisbane-hard,23.0,2017-01-01,dolgopolov_o.2017-01-01,schwartzman_d.2017-01-01
1,2017,1.0,Outdoor,Hard,de_minaur_a.,johnson_s.,28,2017-brisbane-hard,23.0,2017-01-01,de_minaur_a.2017-01-01,johnson_s.2017-01-01
2,2018,5.0,Outdoor,Hard,harrison_r.,mayer_l.,28,2018-brisbane-hard,27.0,2018-01-01,harrison_r.2018-01-01,mayer_l.2018-01-01
3,2018,5.0,Outdoor,Hard,ebden_m.,tiafoe_f.,28,2018-brisbane-hard,17.0,2018-01-01,ebden_m.2018-01-01,tiafoe_f.2018-01-01
4,2018,5.0,Outdoor,Hard,zverev_m.,smith_j.p.,28,2018-brisbane-hard,22.0,2018-01-01,zverev_m.2018-01-01,smith_j.p.2018-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...
5033,2020,3.0,Outdoor,Hard,dimitrov_g.,wawrinka_s.,32,2020-acapulco-hard,20.0,2020-02-27,dimitrov_g.2020-02-27,wawrinka_s.2020-02-27
5034,2020,3.0,Outdoor,Hard,nadal_r.,kwon_s.w.,32,2020-acapulco-hard,15.0,2020-02-27,nadal_r.2020-02-27,kwon_s.w.2020-02-27
5035,2020,2.0,Outdoor,Hard,fritz_t.,isner_j.,32,2020-acapulco-hard,29.0,2020-02-27,fritz_t.2020-02-27,isner_j.2020-02-27
5036,2020,2.0,Outdoor,Hard,nadal_r.,dimitrov_g.,32,2020-acapulco-hard,17.0,2020-02-27,nadal_r.2020-02-27,dimitrov_g.2020-02-27


# Get a unique DataFrame for games from 1991 to 2020


In [7]:
all_matchs_91_2020=pd.concat([match_91_17,match_2018_2020],sort=False).reset_index(drop=True)
#Some players have unknown players, we will remove them. 
all_matchs_91_2020=all_matchs_91_2020[~all_matchs_91_2020['winner'].str.contains('unknown')]
all_matchs_91_2020=all_matchs_91_2020[~all_matchs_91_2020['loser'].str.contains('unknown')]
all_matchs_91_2020.to_csv('jean_work/processed_data/all_match_91_2020.csv',index=False)
 

all_matchs_91_2020

,year,round,winner,loser,games_played,court,surface,tourney_singles_draw,tournoi_unique_id,week_title,id_winner,id_loser
0,1991,1.0,novacek_k.,fleurian_j.,26.0,Outdoor,Hard,32,1991-auckland-hard,1991-01-07,novacek_k.1991-01-07,fleurian_j.1991-01-07
1,1991,2.0,novacek_k.,vajda_m.,26.0,Outdoor,Hard,32,1991-auckland-hard,1991-01-07,novacek_k.1991-01-07,vajda_m.1991-01-07
2,1991,2.0,fleurian_j.,mattar_l.,28.0,Outdoor,Hard,32,1991-auckland-hard,1991-01-07,fleurian_j.1991-01-07,mattar_l.1991-01-07
3,1991,3.0,fleurian_j.,sanchez_e.,21.0,Outdoor,Hard,32,1991-auckland-hard,1991-01-07,fleurian_j.1991-01-07,sanchez_e.1991-01-07
4,1991,3.0,novacek_k.,jonsson_l.,0.0,Outdoor,Hard,32,1991-auckland-hard,1991-01-07,novacek_k.1991-01-07,jonsson_l.1991-01-07
...,...,...,...,...,...,...,...,...,...,...,...,...
106685,2020,3.0,dimitrov_g.,wawrinka_s.,20.0,Outdoor,Hard,32,2020-acapulco-hard,2020-02-27,dimitrov_g.2020-02-27,wawrinka_s.2020-02-27
106686,2020,3.0,nadal_r.,kwon_s.w.,15.0,Outdoor,Hard,32,2020-acapulco-hard,2020-02-27,nadal_r.2020-02-27,kwon_s.w.2020-02-27
106687,2020,2.0,fritz_t.,isner_j.,29.0,Outdoor,Hard,32,2020-acapulco-hard,2020-02-27,fritz_t.2020-02-27,isner_j.2020-02-27
106688,2020,2.0,nadal_r.,dimitrov_g.,17.0,Outdoor,Hard,32,2020-acapulco-hard,2020-02-27,nadal_r.2020-02-27,dimitrov_g.2020-02-27


# Get a Data Frame withe the ranking of each player from 1991 to 2020

In [9]:
ranking_91_17=pd.read_csv('raw_data/rankings_1973_2017_csv.csv')
ranking_91_17['week_title']=ranking_91_17['week_title'].apply(lambda x: x.replace('.','-'))

#Change players name, to make same in the same format than other table
def change_player_slug(player_slug):
    player_slug=player_slug.split('-')
    player_slug=''.join(player_slug[1:])+'_'+player_slug[0][0]+'.'
    return player_slug

ranking_91_17['player_slug']=ranking_91_17['player_slug'].apply(lambda x: change_player_slug(x))
ranking_91_17['unique_id']=ranking_91_17['player_slug']+ranking_91_17['week_title']
ranking_91_17=ranking_91_17[['week_title','week_year','rank_number','ranking_points','tourneys_played','player_slug','unique_id']]
ranking_91_17=ranking_91_17.rename({'player_slug':'player_name','week_year':'year'},axis=1)
ranking_91_17.head()

,week_title,year,rank_number,ranking_points,tourneys_played,player_name,unique_id
0,2017-11-20,2017,1,10645,18,nadal_r.,nadal_r.2017-11-20
1,2017-11-20,2017,2,9605,17,federer_r.,federer_r.2017-11-20
2,2017-11-20,2017,3,5150,23,dimitrov_g.,dimitrov_g.2017-11-20
3,2017-11-20,2017,4,4610,25,zverev_a.,zverev_a.2017-11-20
4,2017-11-20,2017,5,4015,27,thiem_d.,thiem_d.2017-11-20


In [10]:
match_2018_2020=pd.DataFrame()
for year in range (2018,2021):
    match_2018_2020=pd.concat([match_2018_2020,pd.read_excel('raw_data/'+str(year)+'.xlsx')],sort=False)
match_2018_2020=pd.concat([match_2018_2020[match_2018_2020.columns[0:27]],match_2018_2020[match_2018_2020.columns[-2:]]],axis=1)
match_2018_2020['games_played']=match_2018_2020[['W1','W2','W3','W4','W5','L1','L2','L3','L4','L5']].fillna(0).sum(axis=1)
match_2018_2020=match_2018_2020[['Location','Tournament','Date','Round','Court','Surface','Winner','Loser','WRank','LRank','AvgW','AvgL','WPts','LPts','games_played']]

match_2018_2020['Date']=pd.to_datetime(match_2018_2020['Date'])
match_2018_2020['year']=pd.to_datetime(match_2018_2020['Date']).dt.year

#We create a unique key for each tournament, in order to merge it with tournaments dictionnary
match_2018_2020['tournoi_unique_id']=match_2018_2020['year'].apply(lambda x:str(x))+'-'+match_2018_2020['Location'].apply(lambda x:x.lower())+'-'+match_2018_2020['Surface'].apply(lambda x:x.lower())

match_2018_2020=pd.merge(match_2018_2020,dict_tournoi,on='tournoi_unique_id')
match_2018_2020=match_2018_2020[['Date','year','Round','Court','Surface','Winner','Loser','games_played','WRank','LRank','WPts','LPts','tourney_singles_draw','tournoi_unique_id']]
match_2018_2020=match_2018_2020.rename(str.lower,axis='columns')
match_2018_2020['winner']=match_2018_2020['winner'].apply(lambda x:x.replace(' ','_').lower())
match_2018_2020['loser']=match_2018_2020['loser'].apply(lambda x:x.replace(' ','_').lower())

# #Ranking for winners
winner_rank=match_2018_2020.groupby(['tournoi_unique_id','winner','year'])[['winner','wrank','wpts']].mean()
winner_rank=winner_rank.reset_index()[['tournoi_unique_id','winner','year','wrank','wpts']]
winner_rank=winner_rank.rename({'winner':'player_name','wrank':'rank_number','tournoi_unique_id':'week_title','wpts':'ranking_points'},axis=1)


#Ranking for losers
loser_rank=match_2018_2020.groupby(['tournoi_unique_id','loser','year'])[['loser','lrank','lpts']].mean()
loser_rank=loser_rank.reset_index()[['tournoi_unique_id','loser','year','lrank','lpts']]
loser_rank=loser_rank.rename({'loser':'player_name','lrank':'rank_number','tournoi_unique_id':'week_title','lpts':'ranking_points'},axis=1)

#one df for all ranks
ranking_2018_2020=pd.concat([winner_rank,loser_rank])
ranking_2018_2020['week_title']=ranking_2018_2020['week_title'].map(dict_tournoi[['tournoi_unique_id','tourney_dates']].set_index('tournoi_unique_id')['tourney_dates'])
ranking_2018_2020['player_week_id']=ranking_2018_2020['player_name']+'-'+ranking_2018_2020['week_title'].apply(lambda x:str(x))
ranking_2018_2020=ranking_2018_2020[ranking_2018_2020['year']>2017]

#Crete one coumn with the number of tourney played from the beginning of the year
for year in range(2018,2021):
    df_year=ranking_2018_2020[ranking_2018_2020['year']==year].sort_values('week_title')
    for date in np.unique(np.array(df_year['week_title'].values)):
        df=df_year[df_year['week_title']<=date].groupby('player_name').count().reset_index()
        df['player_week_id']=df['player_name']+'-'+str(date)
        map_tournoi=df[['player_week_id','week_title']].set_index('player_week_id')['week_title']
        ranking_2018_2020.loc[ranking_2018_2020['week_title']==date,'tourneys_played']=ranking_2018_2020[ranking_2018_2020['week_title']==date]['player_week_id'].map(map_tournoi)

ranking_2018_2020=ranking_2018_2020.drop('player_week_id',axis=1)
ranking_2018_2020['unique_id']=ranking_2018_2020['player_name']+ranking_2018_2020['week_title']
ranking_2018_2020.head()

,week_title,player_name,year,rank_number,ranking_points,tourneys_played,unique_id
2,2018-02-27,anderson_k.,2018,8.0,2825.0,3.0,anderson_k.2018-02-27
3,2018-02-27,chung_h.,2018,29.0,1497.0,10.0,chung_h.2018-02-27
4,2018-02-27,del_potro_j.m.,2018,9.0,2745.0,7.0,del_potro_j.m.2018-02-27
5,2018-02-27,donaldson_j.,2018,59.0,878.0,9.0,donaldson_j.2018-02-27
6,2018-02-27,ebden_m.,2018,87.0,640.0,7.0,ebden_m.2018-02-27


In [11]:
ranking_all=pd.concat([ranking_91_17,ranking_2018_2020]).sort_values('week_title').reset_index(drop=True)
ranking_all.to_csv('jean_work/processed_data/ranking_91_2020_clean.csv')
ranking_all.head()

,week_title,year,rank_number,ranking_points,tourneys_played,player_name,unique_id
0,1973-08-23,1973,43.0,0.0,0.0,amritraj_v.,amritraj_v.1973-08-23
1,1973-08-23,1973,67.0,0.0,0.0,carmichael_b.,carmichael_b.1973-08-23
2,1973-08-23,1973,66.0,0.0,0.0,gisbert%20sr_j.,gisbert%20sr_j.1973-08-23
3,1973-08-23,1973,65.0,0.0,0.0,battrick_g.,battrick_g.1973-08-23
4,1973-08-23,1973,64.0,0.0,0.0,pasarell_c.,pasarell_c.1973-08-23


Now, let's add a column corresponding to the average ranking of each player for each week

In [12]:
ranking_avg=ranking_all.rename({'week_title':'tourney_dates'},axis=1)
ranking_avg=ranking_avg.sort_values('tourney_dates').reset_index(drop=True)

ranking_avg[['rank_avg','points_avg']] = (ranking_avg.groupby(['player_name'], 
                        as_index=False)['rank_number','ranking_points']
               .expanding().mean()
               .reset_index(level=0, drop=True)
                        )

ranking_avg.to_csv('jean_work/processed_data/ranking_average_all.csv',index=False)
ranking_avg.head()

,tourney_dates,year,rank_number,ranking_points,tourneys_played,player_name,unique_id,rank_avg,points_avg
0,1973-08-23,1973,43.0,0.0,0.0,amritraj_v.,amritraj_v.1973-08-23,43.0,0.0
1,1973-08-23,1973,184.0,0.0,0.0,spear_n.,spear_n.1973-08-23,184.0,0.0
2,1973-08-23,1973,183.0,0.0,0.0,lloyd_d.,lloyd_d.1973-08-23,183.0,0.0
3,1973-08-23,1973,182.0,0.0,0.0,robbins_f.,robbins_f.1973-08-23,182.0,0.0
4,1973-08-23,1973,181.0,0.0,0.0,hose_h.,hose_h.1973-08-23,181.0,0.0


In [13]:
# a bit slow, previous cell does the same thing faster
## means need to stay commented when uncommenting #

# ranking_avg=ranking_all.rename({'week_title':'tourney_dates'},axis=1)
# # ranking_avg['unique_id']=ranking_avg['player_name']+ranking_avg['tourney_dates']
# ranking_avg=ranking_avg.sort_values('tourney_dates').reset_index(drop=True)
# # ranking_avg=ranking_avg.fillna(0)

# for date in np.unique(np.array(ranking_avg['tourney_dates'].values)):
#     df=ranking_avg[ranking_avg['tourney_dates']<=date]
#     df=df.groupby('player_name').mean().reset_index()
#     df['unique_id']=df['player_name']+(date)
#     serie_rank=pd.Series(df[['unique_id','rank_number']].set_index('unique_id')['rank_number'])
#     ranking_avg.loc[ranking_avg['tourney_dates']==date,'rank_avg']=ranking_avg[ranking_avg['tourney_dates']==date]['unique_id'].map(serie_rank.to_dict())
#     serie_rank_points=pd.Series(df[['unique_id','ranking_points']].set_index('unique_id')['ranking_points'])
#     ranking_avg.loc[ranking_avg['tourney_dates']==date,'points_avg']=ranking_avg[ranking_avg['tourney_dates']==date]['unique_id'].map(serie_rank_points.to_dict())
    
# ranking_avg.to_csv('ranking_average_all.csv',index=False)
# ranking_avg.head()


Now we will map each of these columns to the match data_set

In [14]:
ranking_avg=ranking_avg[~ranking_avg['player_name'].str.contains('unknown')]
ranking_avg=ranking_avg.drop_duplicates('unique_id') #drops 12000 rows out of 2.7 million 

map_rank=ranking_avg[['unique_id','rank_number']].set_index('unique_id')['rank_number']
all_matchs_91_2020['wrank']=all_matchs_91_2020['id_winner'].map(map_rank)
all_matchs_91_2020['lrank']=all_matchs_91_2020['id_loser'].map(map_rank)

map_pts=ranking_avg[['unique_id','ranking_points']].set_index('unique_id')['ranking_points']
all_matchs_91_2020['wpts']=all_matchs_91_2020['id_winner'].map(map_pts)
all_matchs_91_2020['lpts']=all_matchs_91_2020['id_loser'].map(map_pts)

map_avg_rank=ranking_avg[['unique_id','rank_avg']].set_index('unique_id')['rank_avg']
all_matchs_91_2020['wrank_avg']=all_matchs_91_2020['id_winner'].map(map_avg_rank)
all_matchs_91_2020['lrank_avg']=all_matchs_91_2020['id_loser'].map(map_avg_rank)

map_avg_pts=ranking_avg[['unique_id','points_avg']].set_index('unique_id')['points_avg']
all_matchs_91_2020['wpts_avg']=all_matchs_91_2020['id_winner'].map(map_avg_pts)
all_matchs_91_2020['lpts_avg']=all_matchs_91_2020['id_loser'].map(map_avg_pts)


all_matchs_91_2020

,year,round,winner,loser,games_played,court,surface,tourney_singles_draw,tournoi_unique_id,week_title,id_winner,id_loser,wrank,lrank,wpts,lpts,wrank_avg,lrank_avg,wpts_avg,lpts_avg
0,1991,1.0,novacek_k.,fleurian_j.,26.0,Outdoor,Hard,32,1991-auckland-hard,1991-01-07,novacek_k.1991-01-07,fleurian_j.1991-01-07,35.0,NaN,0.0,NaN,95.782759,NaN,0.000000,NaN
1,1991,2.0,novacek_k.,vajda_m.,26.0,Outdoor,Hard,32,1991-auckland-hard,1991-01-07,novacek_k.1991-01-07,vajda_m.1991-01-07,35.0,83.0,0.0,0.0,95.782759,91.913793,0.000000,0.000000
2,1991,2.0,fleurian_j.,mattar_l.,28.0,Outdoor,Hard,32,1991-auckland-hard,1991-01-07,fleurian_j.1991-01-07,mattar_l.1991-01-07,NaN,45.0,NaN,0.0,NaN,130.262774,NaN,0.000000
3,1991,3.0,fleurian_j.,sanchez_e.,21.0,Outdoor,Hard,32,1991-auckland-hard,1991-01-07,fleurian_j.1991-01-07,sanchez_e.1991-01-07,NaN,9.0,NaN,0.0,NaN,39.731959,NaN,0.000000
4,1991,3.0,novacek_k.,jonsson_l.,0.0,Outdoor,Hard,32,1991-auckland-hard,1991-01-07,novacek_k.1991-01-07,jonsson_l.1991-01-07,35.0,84.0,0.0,0.0,95.782759,400.751592,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106685,2020,3.0,dimitrov_g.,wawrinka_s.,20.0,Outdoor,Hard,32,2020-acapulco-hard,2020-02-27,dimitrov_g.2020-02-27,wawrinka_s.2020-02-27,22.0,16.0,1682.0,2060.0,174.880909,126.545877,1378.136280,2043.639954
106686,2020,3.0,nadal_r.,kwon_s.w.,15.0,Outdoor,Hard,32,2020-acapulco-hard,2020-02-27,nadal_r.2020-02-27,kwon_s.w.2020-02-27,2.0,76.0,9395.0,732.0,60.065947,101.692308,5999.516787,576.769231
106687,2020,2.0,fritz_t.,isner_j.,29.0,Outdoor,Hard,32,2020-acapulco-hard,2020-02-27,fritz_t.2020-02-27,isner_j.2020-02-27,35.0,20.0,1255.0,1760.0,384.638554,176.014416,530.273092,1461.701062
106688,2020,2.0,nadal_r.,dimitrov_g.,17.0,Outdoor,Hard,32,2020-acapulco-hard,2020-02-27,nadal_r.2020-02-27,dimitrov_g.2020-02-27,2.0,22.0,9395.0,1682.0,60.065947,174.880909,5999.516787,1378.136280


In [15]:
#Let's create a feature corresponding to the avg number of games played before a given game
# Takes 6 min, can be optimized but not trivial
ind = all_matchs_91_2020['tournoi_unique_id'].value_counts().index
for tournoi in tqdm(ind):
    df_tournoi=all_matchs_91_2020[all_matchs_91_2020['tournoi_unique_id']==tournoi]
    for index,row in df_tournoi.iterrows():
        winner=row['winner']
        loser=row['loser']
        df_before=df_tournoi.loc[index+1:,]
        df_winner=df_before[df_before['winner']==winner]
        nb_games_winner=df_winner['games_played'].sum()
        df_loser=df_before[df_before['winner']==loser]
        nb_games_loser=df_loser['games_played'].sum()
        all_matchs_91_2020.loc[index,'winner_games_before']=nb_games_winner
        all_matchs_91_2020.loc[index,'loser_games_before']=nb_games_loser         
        
        
all_matchs_91_2020.head()  

100%|██████████| 2048/2048 [04:02<00:00,  8.45it/s]


,year,round,winner,loser,games_played,court,surface,tourney_singles_draw,tournoi_unique_id,week_title,...,wrank,lrank,wpts,lpts,wrank_avg,lrank_avg,wpts_avg,lpts_avg,winner_games_before,loser_games_before
0,1991,1.0,novacek_k.,fleurian_j.,26.0,Outdoor,Hard,32,1991-auckland-hard,1991-01-07,...,35.0,NaN,0.0,NaN,95.782759,NaN,0.0,NaN,76.0,88.0
1,1991,2.0,novacek_k.,vajda_m.,26.0,Outdoor,Hard,32,1991-auckland-hard,1991-01-07,...,35.0,83.0,0.0,0.0,95.782759,91.913793,0.0,0.0,50.0,76.0
2,1991,2.0,fleurian_j.,mattar_l.,28.0,Outdoor,Hard,32,1991-auckland-hard,1991-01-07,...,NaN,45.0,NaN,0.0,NaN,130.262774,NaN,0.0,60.0,45.0
3,1991,3.0,fleurian_j.,sanchez_e.,21.0,Outdoor,Hard,32,1991-auckland-hard,1991-01-07,...,NaN,9.0,NaN,0.0,NaN,39.731959,NaN,0.0,39.0,49.0
4,1991,3.0,novacek_k.,jonsson_l.,0.0,Outdoor,Hard,32,1991-auckland-hard,1991-01-07,...,35.0,84.0,0.0,0.0,95.782759,400.751592,0.0,0.0,50.0,59.0


# Creation of players profile with games' stats

Now, for each player from 1991, we will create a profile corresponding to his style of play. I didn't use the dataset below in the previouss computation, since it has less games. 

In [16]:
match=pd.DataFrame()
for k in range (1991,2021): 
    df=pd.read_csv('raw_data/atp_matches_'+str(k)+'.csv')
    match=pd.concat([match,df])
    
match_winner=match[[
       'winner_name', 'winner_hand', 'winner_ht',
       'w_ace', 'w_df', 'w_svpt', 'w_1stIn', 'w_1stWon', 'w_2ndWon',
       'w_SvGms', 'w_bpSaved', 'w_bpFaced']]

match_winner=match_winner.fillna(match_winner[['w_ace', 'w_df', 'w_svpt', 'w_1stIn', 'w_1stWon', 'w_2ndWon',
       'w_SvGms', 'w_bpSaved', 'w_bpFaced']].median())

match_loser=match[[
       'loser_name', 'loser_hand', 'loser_ht',
       'l_ace', 'l_df', 'l_svpt', 'l_1stIn', 'l_1stWon', 'l_2ndWon',
       'l_SvGms', 'l_bpSaved', 'l_bpFaced']]
match_loser=match_loser.fillna(match_loser[['l_ace', 'l_df', 'l_svpt', 'l_1stIn', 'l_1stWon', 'l_2ndWon',
       'l_SvGms', 'l_bpSaved', 'l_bpFaced']].median())

match_winner['player_name']=match_winner['winner_name'].apply(lambda x:change_player(x)).str.lower()
match_winner['percentage_aces']=match_winner['w_ace']/match_winner['w_svpt']
match_winner['percentage_df']=match_winner['w_df']/match_winner['w_svpt']
match_winner['percentage_1stwon']=match_winner['w_1stWon']/match_winner['w_1stIn']
match_winner['percentage_bpsaved']=match_winner['w_bpSaved']/match_winner['w_bpFaced']
match_winner['percentage_bpobtained']=match_loser['l_bpFaced']/match_loser['l_svpt']
match_winner['percentage_bpconverted']=1-(match_loser['l_bpSaved']/match_loser['l_bpFaced'])

match_loser['player_name']=match_loser['loser_name'].apply(lambda x:change_player(x)).str.lower()
match_loser['percentage_aces']=match_loser['l_ace']/match_loser['l_svpt']
match_loser['percentage_df']=match_loser['l_df']/match_loser['l_svpt']
match_loser['percentage_1stwon']=match_loser['l_1stWon']/match_loser['l_1stIn']
match_loser['percentage_bpsaved']=match_loser['l_bpSaved']/match_loser['l_bpFaced']
match_loser['percentage_bpobtained']=match_winner['w_bpFaced']/match_winner['w_svpt']
match_loser['percentage_bpconverted']=1-(match_winner['w_bpSaved']/match_winner['w_bpFaced'])


match_loser=match_loser[['player_name',
       'percentage_aces', 'percentage_df', 'percentage_1stwon',
       'percentage_bpsaved','percentage_bpobtained','percentage_bpconverted']]
match_winner=match_winner[['player_name',
       'percentage_aces', 'percentage_df', 'percentage_1stwon',
       'percentage_bpsaved','percentage_bpobtained','percentage_bpconverted']]

player_stats=pd.concat([match_winner,match_loser]).groupby('player_name').mean().reset_index()
player_stats

,player_name,percentage_aces,percentage_df,percentage_1stwon,percentage_bpsaved,percentage_bpobtained,percentage_bpconverted
0,aarum_f.,0.053333,0.040000,0.659091,0.500000,0.054795,0.250000
1,abadia_a.,0.057665,0.036399,0.685065,0.571429,0.069615,0.321429
2,abdel nour_b.,0.068493,0.027397,0.750000,0.750000,0.106667,0.500000
3,abdukhalikov_s.,0.060913,0.033699,0.704545,0.625000,0.080731,0.375000
4,abdulaal_e.,0.060913,0.033699,0.704545,0.625000,0.080731,0.375000
...,...,...,...,...,...,...,...
3224,zuk_k.,0.068308,0.027876,0.728090,0.788462,0.092511,0.269231
3225,zunino_j.,0.053333,0.040000,0.659091,0.500000,0.054795,0.250000
3226,zverev_a.,0.102945,0.051035,0.739790,0.592829,0.092355,0.458120
3227,zverev_m.,0.065789,0.029169,0.688218,0.585289,0.075217,0.398262


In [17]:
players=pd.read_csv('raw_data/player_overviews_unindexed_csv.csv')
    
players['player_name']=players['player_slug'].apply(lambda x:change_player_slug(x))
players=players[['birthdate','turned_pro','weight_kg','height_cm','handedness','backhand','player_name']]
players_df=players.merge(player_stats,how='left',on='player_name')
players_df=players_df.drop_duplicates('player_name').set_index('player_name')


players_df.to_csv('jean_work/processed_data/full_dataset_players.csv',index=True)
players_df

,birthdate,turned_pro,weight_kg,height_cm,handedness,backhand,percentage_aces,percentage_df,percentage_1stwon,percentage_bpsaved,percentage_bpobtained,percentage_bpconverted
player_name,,,,,,,,,,,,
acuna_r.,1958.01.13,0.0,68.0,175.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
abdullahi_s.,1960.02.02,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
aerts_n.,1963.04.25,0.0,75.0,188.0,NaN,NaN,0.021681,0.070302,0.695555,0.738636,0.063853,0.350000
adams_e.,1959.06.15,0.0,73.0,178.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
agenor_r.,1964.11.13,1983.0,82.0,180.0,NaN,NaN,0.042478,0.024860,0.651785,0.536894,0.088836,0.438804
...,...,...,...,...,...,...,...,...,...,...,...,...
zahar_n.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
zorab_a.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
zinn_g.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
for columns in players_df.columns:
    for player in ['winner','loser']:
        serie_map=players_df[columns]
        all_matchs_91_2020[player+'_'+columns]=all_matchs_91_2020[player].map(serie_map)
        
all_matchs_91_2020.head()

,year,round,winner,loser,games_played,court,surface,tourney_singles_draw,tournoi_unique_id,week_title,...,winner_percentage_df,loser_percentage_df,winner_percentage_1stwon,loser_percentage_1stwon,winner_percentage_bpsaved,loser_percentage_bpsaved,winner_percentage_bpobtained,loser_percentage_bpobtained,winner_percentage_bpconverted,loser_percentage_bpconverted
0,1991,1.0,novacek_k.,fleurian_j.,26.0,Outdoor,Hard,32,1991-auckland-hard,1991-01-07,...,0.028913,NaN,0.73208,NaN,0.608743,NaN,0.093075,NaN,0.40795,NaN
1,1991,2.0,novacek_k.,vajda_m.,26.0,Outdoor,Hard,32,1991-auckland-hard,1991-01-07,...,0.028913,0.047982,0.73208,0.617429,0.608743,0.511016,0.093075,0.098631,0.40795,0.438917
2,1991,2.0,fleurian_j.,mattar_l.,28.0,Outdoor,Hard,32,1991-auckland-hard,1991-01-07,...,NaN,0.027940,NaN,0.688904,NaN,0.598320,NaN,0.081301,NaN,0.412607
3,1991,3.0,fleurian_j.,sanchez_e.,21.0,Outdoor,Hard,32,1991-auckland-hard,1991-01-07,...,NaN,0.017640,NaN,0.683268,NaN,0.582718,NaN,0.090854,NaN,0.421758
4,1991,3.0,novacek_k.,jonsson_l.,0.0,Outdoor,Hard,32,1991-auckland-hard,1991-01-07,...,0.028913,0.018601,0.73208,0.624786,0.608743,0.543639,0.093075,0.094225,0.40795,0.447947


# Feature creation that could be interesting and used in algorithms

In [20]:
#First, let's keep only interesting features
df_1=all_matchs_91_2020[[ 'year','round', 'winner', 'loser', 'games_played', 'court', 'surface',
       'tourney_singles_draw', 'tournoi_unique_id', 'week_title', 'wrank', 'lrank', 'wpts', 'lpts', 'wrank_avg', 'lrank_avg',
       'wpts_avg', 'lpts_avg', 'winner_percentage_aces',
       'winner_percentage_df', 'winner_percentage_1stwon',
       'winner_percentage_bpsaved', 'winner_percentage_bpobtained',
       'winner_percentage_bpconverted', 'loser_percentage_aces',
       'loser_percentage_df', 'loser_percentage_1stwon',
       'loser_percentage_bpsaved', 'loser_percentage_bpobtained',
       'loser_percentage_bpconverted', 'winner_birthdate', 'loser_birthdate',
       'winner_turned_pro', 'loser_turned_pro', 'winner_weight_kg',
       'loser_weight_kg', 'winner_height_cm', 'loser_height_cm',
       'winner_handedness', 'loser_handedness', 'winner_backhand',
       'loser_backhand','winner_games_before', 'loser_games_before']]

df_1['week_title']=pd.to_datetime(df_1['week_title'])
df_1['winner_birthdate']=pd.to_datetime(df_1['winner_birthdate'])
df_1['loser_birthdate']=pd.to_datetime(df_1['loser_birthdate'])

df_1['winner_age']=(df_1['week_title']-df_1['winner_birthdate'])
df_1['loser_age']=df_1['week_title']-df_1['loser_birthdate']
df_1['winner_year_pro']=df_1['week_title'].dt.year-df_1['winner_turned_pro']
df_1['loser_year_pro']=df_1['week_title'].dt.year-df_1['loser_turned_pro']

df_1=df_1[['year','week_title','round', 'winner', 'loser', 'court', 'surface',
       'tourney_singles_draw', 'tournoi_unique_id', 'wrank', 'lrank', 'wpts', 'lpts',
       'wrank_avg', 'lrank_avg', 'wpts_avg', 'lpts_avg',
       'winner_percentage_aces', 'winner_percentage_df',
       'winner_percentage_1stwon', 'winner_percentage_bpsaved',
       'winner_percentage_bpobtained', 'winner_percentage_bpconverted',
       'loser_percentage_aces', 'loser_percentage_df',
       'loser_percentage_1stwon', 'loser_percentage_bpsaved',
       'loser_percentage_bpobtained', 'loser_percentage_bpconverted', 'winner_weight_kg', 'loser_weight_kg',
       'winner_height_cm', 'loser_height_cm', 'winner_handedness',
       'loser_handedness', 'winner_backhand', 'loser_backhand',
       'winner_games_before', 'loser_games_before', 'winner_age', 'loser_age',
       'winner_year_pro', 'loser_year_pro']]
df_1

,year,week_title,round,winner,loser,court,surface,tourney_singles_draw,tournoi_unique_id,wrank,...,winner_handedness,loser_handedness,winner_backhand,loser_backhand,winner_games_before,loser_games_before,winner_age,loser_age,winner_year_pro,loser_year_pro
0,1991,1991-01-07,1.0,novacek_k.,fleurian_j.,Outdoor,Hard,32,1991-auckland-hard,35.0,...,NaN,NaN,NaN,NaN,76.0,88.0,9414 days,NaT,7.0,NaN
1,1991,1991-01-07,2.0,novacek_k.,vajda_m.,Outdoor,Hard,32,1991-auckland-hard,35.0,...,NaN,NaN,NaN,NaN,50.0,76.0,9414 days,9420 days,7.0,NaN
2,1991,1991-01-07,2.0,fleurian_j.,mattar_l.,Outdoor,Hard,32,1991-auckland-hard,NaN,...,NaN,NaN,NaN,NaN,60.0,45.0,NaT,10004 days,NaN,6.0
3,1991,1991-01-07,3.0,fleurian_j.,sanchez_e.,Outdoor,Hard,32,1991-auckland-hard,NaN,...,NaN,NaN,NaN,NaN,39.0,49.0,NaT,9354 days,NaN,7.0
4,1991,1991-01-07,3.0,novacek_k.,jonsson_l.,Outdoor,Hard,32,1991-auckland-hard,35.0,...,NaN,NaN,NaN,NaN,50.0,59.0,9414 days,7499 days,7.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106685,2020,2020-02-27,3.0,dimitrov_g.,wawrinka_s.,Outdoor,Hard,32,2020-acapulco-hard,22.0,...,Right-Handed,Right-Handed,One-Handed Backhand,One-Handed Backhand,0.0,0.0,10514 days,12754 days,12.0,18.0
106686,2020,2020-02-27,3.0,nadal_r.,kwon_s.w.,Outdoor,Hard,32,2020-acapulco-hard,2.0,...,Left-Handed,NaN,Two-Handed Backhand,NaN,34.0,0.0,12322 days,NaT,19.0,NaN
106687,2020,2020-02-27,2.0,fritz_t.,isner_j.,Outdoor,Hard,32,2020-acapulco-hard,35.0,...,Right-Handed,Right-Handed,Two-Handed Backhand,Two-Handed Backhand,0.0,0.0,8157 days,12725 days,5.0,13.0
106688,2020,2020-02-27,2.0,nadal_r.,dimitrov_g.,Outdoor,Hard,32,2020-acapulco-hard,2.0,...,Left-Handed,Right-Handed,Two-Handed Backhand,One-Handed Backhand,17.0,0.0,12322 days,10514 days,19.0,12.0


Now it is required to find a way to shuffle winner and losers

In [21]:
loser_list=['lrank', 'lpts', 'lrank_avg', 'lpts_avg', 'loser_percentage_aces', 'loser_percentage_df',
       'loser_percentage_1stwon', 'loser_percentage_bpsaved',
       'loser_percentage_bpobtained', 'loser_percentage_bpconverted', 'loser_percentage_aces','loser_weight_kg', 
       'loser_height_cm', 'loser_handedness','loser_backhand', 'loser_games_before', 'loser_age', 'loser_year_pro']
winner_list=['wrank', 'wpts', 'wrank_avg', 'wpts_avg','winner_percentage_aces', 'winner_percentage_df',
       'winner_percentage_1stwon', 'winner_percentage_bpsaved',
       'winner_percentage_bpobtained', 'winner_percentage_bpconverted',
       'winner_percentage_aces','winner_weight_kg', 'winner_height_cm', 'winner_handedness', 'winner_backhand', 
       'winner_games_before', 'winner_age', 'winner_year_pro']

def shuffle_game(row): 
    rand=random()
    if rand>0.5:
        winner=1
        loser=0
        row['winner_player']=1
        return row
    else:
        winner=0
        loser=1
        memory=row[loser_list].values
        row[loser_list]=row[winner_list].values
        row[winner_list]=memory
        row['winner_player']=0
        winner_name=row['winner']
        row['winner']=row['loser']
        row['loser']=winner_name
        return row
        
df_1=df_1.apply(lambda x:shuffle_game(x),axis=1)
df_1.to_csv('jean_work/processed_data/dataset_v1.csv',index=False)
df_1.head()

,year,week_title,round,winner,loser,court,surface,tourney_singles_draw,tournoi_unique_id,wrank,...,loser_handedness,winner_backhand,loser_backhand,winner_games_before,loser_games_before,winner_age,loser_age,winner_year_pro,loser_year_pro,winner_player
0,1991,1991-01-07,1.0,novacek_k.,fleurian_j.,Outdoor,Hard,32,1991-auckland-hard,35.0,...,NaN,NaN,NaN,76.0,88.0,9414 days,NaT,7.0,NaN,1
1,1991,1991-01-07,2.0,vajda_m.,novacek_k.,Outdoor,Hard,32,1991-auckland-hard,83.0,...,NaN,NaN,NaN,76.0,50.0,9420 days,9414 days,NaN,7.0,0
2,1991,1991-01-07,2.0,mattar_l.,fleurian_j.,Outdoor,Hard,32,1991-auckland-hard,45.0,...,NaN,NaN,NaN,45.0,60.0,10004 days,NaT,6.0,NaN,0
3,1991,1991-01-07,3.0,sanchez_e.,fleurian_j.,Outdoor,Hard,32,1991-auckland-hard,9.0,...,NaN,NaN,NaN,49.0,39.0,9354 days,NaT,7.0,NaN,0
4,1991,1991-01-07,3.0,novacek_k.,jonsson_l.,Outdoor,Hard,32,1991-auckland-hard,35.0,...,NaN,NaN,NaN,50.0,59.0,9414 days,7499 days,7.0,3.0,1


In [20]:
df_1.columns


Index(['year', 'week_title', 'round', 'winner', 'loser', 'court', 'surface',
       'tourney_singles_draw', 'wrank', 'lrank', 'wpts', 'lpts', 'wrank_avg',
       'lrank_avg', 'wpts_avg', 'lpts_avg', 'winner_percentage_aces',
       'winner_percentage_df', 'winner_percentage_1stwon',
       'winner_percentage_bpsaved', 'winner_percentage_bpobtained',
       'winner_percentage_bpconverted', 'loser_percentage_aces',
       'loser_percentage_df', 'loser_percentage_1stwon',
       'loser_percentage_bpsaved', 'loser_percentage_bpobtained',
       'loser_percentage_bpconverted', 'winner_weight_kg', 'loser_weight_kg',
       'winner_height_cm', 'loser_height_cm', 'winner_handedness',
       'loser_handedness', 'winner_backhand', 'loser_backhand',
       'winner_games_before', 'loser_games_before', 'winner_age', 'loser_age',
       'winner_year_pro', 'loser_year_pro', 'winner_player'],
      dtype='object')

In [21]:
df_1['winner_percentage_1stwon'].isna().sum()

10433